In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy
import lightgbm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score

In [3]:
dfB = pd.read_parquet('benign_2312_HTML.parquet')


filtered_dfB = dfB.filter(like='html', axis=1)

filtered_dfB['label'] = 'benign'
# Display the filtered DataFrame
print(filtered_dfB['label'])

0         benign
1         benign
2         benign
3         benign
4         benign
           ...  
171217    benign
171218    benign
171219    benign
171220    benign
171221    benign
Name: label, Length: 171222, dtype: object


/tmp/ipykernel_968/655447292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfB['label'] = 'benign'


In [4]:
dfP = pd.read_parquet('phishing_2406_strict_HTML.parquet')
filtered_dfP = dfP.filter(like='html', axis=1)
#filtered_dfP = filtered_dfP.add_prefix('html_')
filtered_dfP['label'] = 'phishing'
filtered_dfP

/tmp/ipykernel_968/329044350.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfP['label'] = 'phishing'


,html_create_element,html_write,html_char_code_at,html_concat,html_escape,html_eval,html_exec,html_from_char_code,html_link,html_parse_int,...,html_num_of_form_js,html_malicious_form,html_most_common,html_num_of_css_internal,html_num_of_css_external,html_num_of_anchors_to_content,html_num_of_anchors_to_void,html_num_of_blank_spaces,html_blocked_keywords_label,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,91,0,phishing
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,21,0,phishing
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17,0,phishing
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,166,0,phishing
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,0,phishing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,53,0,phishing
24583,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,21,0,phishing
24584,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4045,1,phishing
24585,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,239,0,phishing


In [5]:
filtered_dfB.dropna(inplace=True)
filtered_dfP.dropna(inplace=True)
combined_df = pd.concat([filtered_dfB, filtered_dfP], axis=0)

/tmp/ipykernel_968/1271725341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfP.dropna(inplace=True)


In [6]:
combined_df

,html_create_element,html_write,html_char_code_at,html_concat,html_escape,html_eval,html_exec,html_from_char_code,html_link,html_parse_int,...,html_num_of_form_js,html_malicious_form,html_most_common,html_num_of_css_internal,html_num_of_css_external,html_num_of_anchors_to_content,html_num_of_anchors_to_void,html_num_of_blank_spaces,html_blocked_keywords_label,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,benign
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,0,benign
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,422,0,benign
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,benign
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,0,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,53,0,phishing
24583,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,21,0,phishing
24584,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4045,1,phishing
24585,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,239,0,phishing


In [7]:
bool_cols = combined_df.select_dtypes(include='bool').columns
print(bool_cols)
combined_df[bool_cols] = combined_df[bool_cols].astype(int)

pdaFeatures = combined_df.loc[:, combined_df.columns != "label"] # get features
pdaLabel = pd.DataFrame() 
pdaLabel["label"] = combined_df["label"] # get labels

print(len(pdaFeatures))

Index(['html_no_hrefs'], dtype='object')
195809


In [8]:
dX_train, X_test, y_train, y_test = train_test_split(pdaFeatures, pdaLabel, test_size=0.20, random_state=42) # split data into training and testing data
ratio = (y_train["label"] == 0).sum() / (y_train["label"] == 1).sum()

/tmp/ipykernel_968/3603219133.py:2: RuntimeWarning: invalid value encountered in scalar divide
  ratio = (y_train["label"] == 0).sum() / (y_train["label"] == 1).sum()


In [9]:
lightGBMClas = lightgbm.LGBMClassifier(
    objective="binary",
    is_unbalance=True,
    max_depth=13,
    num_leaves=261,
    n_estimators=1498,
    lambda_l1=0.0017993225169621843,
    lambda_l2=0.0006301826170072739,
    bagging_fraction=0.16163071559346373,
    bagging_freq=0,
    max_bin=800,
    learning_rate=0.05437419665350978,
    min_child_samples=3,
    boosting_type="gbdt",
    min_split_gain=0.000010284197711702238,
    pos_bagging_fraction=0.8451744872078454,
    neg_bagging_fraction=0.8714695064033464,
    colsample_bytree=0.42342506718022915,
    verbosity=-1,
    seed = 500
) # set hyperparameters
npLabel: numpy.ndarray = y_train.to_numpy() 
lightGBMClas.fit(dX_train, npLabel.ravel()) # fit model
prediction = lightGBMClas.predict(X_test)
print("F1 score: ", f1_score(y_test, prediction,pos_label='phishing'))
print("Accuracy: ", accuracy_score(y_test, prediction))
print("Balanced accuracy: ", balanced_accuracy_score(y_test, prediction))

F1 score:  0.9524501661129569
Accuracy:  0.9883049895306675
Balanced accuracy:  0.9680644794313671


In [10]:
import joblib
joblib.dump(lightGBMClas, "models/phishing_html_lgbm_model_updated.joblib")

['phishing_html_lgbm_model_updated.joblib']